## CGAE Stage Classification for Task 2

### For today I have the following results we can look at:
### BRCA, GE and ME
### BRCA, GE and GCN
### LUAD, GE and GCN
### LUSC, GE and GCN

#### I unfortunately have an error with STAD data in CGAE, will look into it


In [3]:
import numpy as np
import pandas as pd
from mord import OrdinalRidge
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, precision_recall_fscore_support, matthews_corrcoef, confusion_matrix

In [4]:
# From baseline

def evaluate_classification(y_true, y_pred):
    # returns accuracy, precision, recall, f1, mcc, confusion_matrix

    acc = accuracy_score(y_true, y_pred)
    pr, rc, f1, _ = precision_recall_fscore_support(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    confMat = confusion_matrix(y_true, y_pred)

    return [acc, pr, rc, f1, mcc, confMat]

In [5]:
folder = "/home/bram/jointomicscomp/results/brca2_geme_CGAE 14-09-2021 08:40:31"
ctype = "BRCA"

In [6]:
x_ctype_train_file = "/home/bram/jointomicscomp/data/{}/{}_GE_train.npy".format(ctype, ctype)
y_ctype_train_file = "/home/bram/jointomicscomp/data/{}/{}_ME_train.npy".format(ctype, ctype)
x_ctype_valid_file = "/home/bram/jointomicscomp/data/{}/{}_GE_valid.npy".format(ctype, ctype)
y_ctype_valid_file = "/home/bram/jointomicscomp/data/{}/{}_ME_valid.npy".format(ctype, ctype)
x_ctype_test_file = "/home/bram/jointomicscomp/data/{}/{}_GE_test.npy".format(ctype, ctype)
y_ctype_test_file = "/home/bram/jointomicscomp/data/{}/{}_ME_test.npy".format(ctype, ctype)

# For latent feature extraction
GEtrainctype = np.load(x_ctype_train_file)
GEvalidctype = np.load(x_ctype_valid_file)
GEtestctype = np.load(x_ctype_test_file)

In [7]:
y_train = np.load("/home/bram/jointomicscomp/data/{}/{}_train_stageType.npy".format(ctype, ctype))
y_valid = np.load("/home/bram/jointomicscomp/data/{}/{}_valid_stageType.npy".format(ctype, ctype))
y_test = np.load("/home/bram/jointomicscomp/data/{}/{}_test_stageType.npy".format(ctype, ctype))



In [8]:
# Z1 for GE and z2 for ME
z1 = np.load("{}/CGAE/task2_z1.npy".format(folder))
z2 = np.load("{}/CGAE/task2_z2.npy".format(folder))

In [9]:
# Create labels from stageTypes and stageTypes
latent_train1 = z1[:len(GEtrainctype)]
latent_valid1 = z1[len(latent_train1):(len(GEtrainctype) + len(GEvalidctype))]
latent_test1 = z1[(len(latent_train1) + len(latent_valid1)):(len(GEtrainctype) + len(GEvalidctype) + len(GEtestctype))]

print(latent_train1.shape)
print(latent_valid1.shape)
print(latent_test1.shape)

latent_train2 = z2[:len(GEtrainctype)]
latent_valid2 = z2[len(latent_train2):(len(GEtrainctype) + len(GEvalidctype))]
latent_test2 = z2[(len(latent_train2) + len(latent_valid2)):(len(GEtrainctype) + len(GEvalidctype) + len(GEtestctype))]

print(latent_train2.shape)
print(latent_valid2.shape)
print(latent_test2.shape)

(608, 128)
(68, 128)
(76, 128)
(608, 128)
(68, 128)
(76, 128)


In [10]:
np.concatenate((y_train, y_valid)).shape

(676,)

In [2]:
print("RAW DATA: Baseline predictions")
alphas = np.array([1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1.0, 2.0, 5.0, 10., 20.])

# if criterion == 'acc':
#     ind = 0
# elif criterion == 'pr':
#     ind = 1
# elif criterion == 'rc':
#     ind = 2
# elif criterion == 'f1':
#     ind = 3
# else:
#     assert criterion == 'mcc'
#     ind = 4

latent_train = np.load("/home/bram/jointomicscomp/data/task1/GE_train.npy")
latent_val = np.load("/home/bram/jointomicscomp/data/task1/GE_val.npy")
latent_test = np.load("/home/bram/jointomicscomp/data/task1/GE_test.npy")
y_train = cancerTypes[np.load("/home/bram/jointomicscomp/data/task1/cancerType_train.npy").astype(int)]
y_val = cancerTypes[np.load("/home/bram/jointomicscomp/data/task1/cancerType_val.npy").astype(int)]
y_test = cancerTypes[np.load("/home/bram/jointomicscomp/data/task1/cancerType_test.npy").astype(int)]



validationPerformance = np.zeros(alphas.shape[0])
models = []

for i, a in enumerate(alphas):
    model = OrdinalRidge(alpha=a, fit_intercept=True, normalize=False, random_state=1)

    # train
    model.fit(latent_train, y_train)

    # save so that we don't have to re-train
    models.append(model)

    # evaluate using user-specified criterion
    validationPerformance[i] = evaluate_classification(y_valid, model.predict(latent_valid))[0]

bestModel = models[np.argmax(validationPerformance)]

predictions = bestModel.predict(latent_test).astype(int)

classifications1 = evaluate_classification(y_test, predictions)



RAW DATA: Baseline predictions


NameError: name 'np' is not defined

In [15]:
print("Accuracy : ", classifications1[0])
print("Confusion matrix : \n", classifications1[5])

Accuracy :  0.5657894736842105
Confusion matrix : 
 [[ 0 12  0  0]
 [ 0 43  0  0]
 [ 0 20  0  0]
 [ 0  1  0  0]]


In [16]:
print("CGAE: Stage predictions from ME using cancer type {}".format(ctype))
alphas = np.array([1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1.0, 2.0, 5.0, 10., 20.])

# if criterion == 'acc':
#     ind = 0
# elif criterion == 'pr':
#     ind = 1
# elif criterion == 'rc':
#     ind = 2
# elif criterion == 'f1':
#     ind = 3
# else:
#     assert criterion == 'mcc'
#     ind = 4



validationPerformance = np.zeros(alphas.shape[0])
models = []

for i, a in enumerate(alphas):
    model = OrdinalRidge(alpha=a, fit_intercept=True, normalize=False, random_state=1)

    # train
    model.fit(latent_train2, y_train)

    # save so that we don't have to re-train
    models.append(model)

    # evaluate using user-specified criterion
    validationPerformance[i] = evaluate_classification(y_valid, model.predict(latent_valid2))[0]

bestModel = models[np.argmax(validationPerformance)]

predictions = bestModel.predict(latent_test2).astype(int)

classifications2 = evaluate_classification(y_test, predictions)



CGAE: Stage predictions from ME using cancer type BRCA


/home/bram/Documents/anaconda3/envs/vae/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bram/Documents/anaconda3/envs/vae/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bram/Documents/anaconda3/envs/vae/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [17]:
print("Accuracy : ", classifications2[0])
print("Confusion matrix : \n", classifications2[5])

Accuracy :  0.5657894736842105
Confusion matrix : 
 [[ 0 12  0  0]
 [ 0 43  0  0]
 [ 0 20  0  0]
 [ 0  1  0  0]]
